In [36]:
from collections import defaultdict
import pandas as pd
from surprise import SVD
from surprise import Dataset

In [37]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [38]:
# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [39]:
top_n = get_top_n(predictions, n=15)

In [40]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    if uid == '668':
        movie_list = [iid for (iid, _) in user_ratings]
        print(uid, [iid for (iid, _) in user_ratings])

668 ['408', '127', '483', '178', '603', '320', '657', '169', '488', '168', '275', '12', '185', '493', '114']


In [41]:
df_sel = pd.DataFrame( movie_list )
df_sel.columns = ['movieId']
df_sel['movieId'] = df_sel['movieId'].astype(int)

In [42]:
df_mov = pd.read_csv( '../movielens_small/movies.csv' )

In [44]:
df_userp = df_sel.join(df_mov.set_index('movieId'), on = 'movieId' )
df_userp.head(10)

,movieId,title,genres
0,408,8 Seconds (1994),Drama
1,127,NaN,NaN
2,483,King of the Hill (1993),Drama
3,178,Love & Human Remains (1993),Comedy|Drama
4,603,"Bye Bye, Love (1995)",Comedy
5,320,Suture (1993),Film-Noir|Thriller
6,657,NaN,NaN
7,169,Free Willy 2: The Adventure Home (1995),Adventure|Children|Drama
8,488,M. Butterfly (1993),Drama|Romance
9,168,First Knight (1995),Action|Drama|Romance
